---
# Praxisteil 4 - Metaheuristiken
UEbergeordnete Loesung
注意，老师有最优答案，因此在beleg中可以多尝试几种不同的方法得出结果，这就要求对5次课的内容熟练掌握。今晚整理简书~~
---
写的代码要可理解，注意变量命名

毕业论文可以跟buscher写，python就够了，不需要为了代码效率学c

下周分配任务，一小时即可

去年的成绩很好

---
## 1. Iterated Greedy
---

---
### 1.1 Wiederholung allgemeiner Ablauf
---

Ablauf Iterated Greedy-->2个原则，结构和构建

![AblaufIG](../DataFiles/IteratedGreedyAblauf.png)

----
### 1.2 Startlösung
----
NEH-Heuristik

In [9]:
from Solver import *

data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json")

solver = Solver(data, 1060) #1014, 754 |  1060, 736     736 已经是优化的结果了，第五中，会出现两个一样的结果

startSolution = solver.ConstructionPhase("NEH")  # 用neh创建初始方案

# 用deepcopy这样被修改的就不是start，而是current。这一步的目的是把初始方案作为当前方案，
# 后面如果发现更好的方案，则把这个更好的方案作为current；经过了deep copy后，我们最初建立的方案数据不会消失
# 后面可以用初始方案作比较
currentSolution = deepcopy(startSolution) 


Generating an initial solution according to NEH.
Constructive solution found.
The permutation [4, 3, 7, 1, 8, 2, 9, 0, 5, 6] results in a Makespan of 752


----
### 1.3 Destruktion
----
Entferne zufällig _numberJobsToRemove_ Aufträge aus aktueller Lösung. Dabei ist _numberJobsToRemove_ ein Parameter des Iterated Greedy.
Als Ergebnis ensteht die Menge entfernter Aufträge _removedJobs_ und eine unvollständige Lösung _partialPermutation_.

In [10]:
numberJobsToRemove = 2 # 1st parameter

def Destruction(solution):
    # 思考：怎么随机删除？？？？
    # choice() 方法返回一个列表，元组或字符串的随机项。注意：choice()是不能直接访问的，需要导入 random 模块，然后通过 random 静态对象调用该方法。
    removedJobs = solver.RNG.choice(solver.InputData.n, size = numberJobsToRemove, replace=False).tolist() # 有放回和无放回。InputData.n是jobs的数量

    partialPermutation = [ i for i in solution.Permutation if i not in removedJobs]

    return removedJobs, partialPermutation # 两个返回值，也就是说，调用该函数的时候需要两个变量

# 使用解构函数把currentSolution分解成两部分
removedJobs, partialPermutation = Destruction(currentSolution)

print(f'removed jobs: {removedJobs}')
print(f'partial permutation: {partialPermutation}')

removed jobs: [0, 9]
partial permutation: [4, 3, 7, 1, 8, 2, 5, 6]


In [11]:
help(random.choice)

Help on method choice in module random:

choice(seq) method of random.Random instance
    Choose a random element from a non-empty sequence.



----
### 1.4 Konstruktion
----
Füge die _removedJobs_ unter Beachtung der Reihenfolge an der jeweils besten Position (NEH) wieder zu _partialPermutation_ hinzu und gib die neue vollständige Lösung zurück. 

In [12]:
def Construction(removedJobs, partialPermutation):
    completeSolution = Solution(solver.InputData.InputJobs, partialPermutation)  # completeSolution在这里还没有complete
    
    # 注意： 重构时要把删除的元素逐一插入回最优位置，因此要用for循环
    for i in removedJobs:
        solver.EvaluationLogic.DetermineBestInsertion(completeSolution, i) # best insertion in neh

    return completeSolution

# newSolution现在是完整的
newSolution = Construction(removedJobs, partialPermutation)
print(f'new complete solution after construction: \n{newSolution}')



new complete solution after construction: 
The permutation [4, 3, 0, 7, 1, 8, 2, 9, 5, 6] results in a Makespan of 736


In [13]:
print(newSolution.Makespan)

736


----
### 1.5 Akzeptanz und Gedächtnis
---- 
Akzeptiere neue Lösung als aktuelle Lösung, wenn
* neue Lösung besser als aktuelle Lösung (immer)
* Akzeptanzkriterium erfüllt (zufällig)

注意看公式中的数字关系，根据幂函数来判断
Akzeptanzkriterium bei Ruiz/ Stützle:  $\pi''$ - neue Lösung (_newSolution_), $\pi$ - aktuelle Lösung (_currentSolution_)
$$
    \begin{equation}
    e^{- \frac{C_{\max}\left( \pi'' \right) - C_{\max}\left( \pi \right)}{Temperature}} \text{ mit } Temperature = T \cdot \frac{\sum Bearbeitungszeiten}{n \cdot m \cdot 10}
    \end{equation}
$$  
$T$ ist der 2. Parameter des Iterated Greedy. Je höher $T$ ist, desto größer ist die Wahrscheinlichkeit, schlechtere Lösungen zu akzeptieren.

Neue beste Lösung wird im _SolutionPool_ gespeichert.

In [14]:
print(f'Solution before checking for acceptance: \n{newSolution}')

baseTemperature = 1 # 2nd parameter
currentBestObj = solver.SolutionPool.GetLowestMakespanSolution().Makespan

if newSolution.Makespan < currentSolution.Makespan: # 若新方案比当下方案更短，则把新方案作为当下方案
    currentSolution = newSolution #接受新的解

    if newSolution.Makespan < currentBestObj: # 更近一步，若新方案比目标函数还要短，则新方案的时长作为目标函数值
        solver.SolutionPool.AddSolution(newSolution)
        currentBestObj = newSolution.Makespan

else:
    newObjValue = newSolution.Makespan
    currentObjValue = currentSolution.Makespan

    totalProcessingTime = sum(x.ProcessingTime(i)   # 公式T中的分子
    for x in solver.InputData.InputJobs 
    for i in range(len(x.Operations))) # add to Inputdata.py to save calls


    temperature = baseTemperature * totalProcessingTime / (solver.InputData.n * solver.InputData.m * 10) # n是订单数，m是机器数
    probability  = math.exp(-(newObjValue - currentBestObj) / temperature)  # wsk，看公式
    # 思考：若两个Obj结果相同，会接受吗？概率是多少？？

# 这里为什么要用随机数跟概率作比较？？？
    randomNumber = solver.RNG.random()  # self.RNG = numpy.random.default_rng(self.Seed) Solver中的RNG实际上是个随机数生成器，要想生成随机数还需要random函数
    if randomNumber <= probability:
        currentSolution = newSolution
        
print(f'Solution after checking for acceptance: \n{currentSolution}')

Solution before checking for acceptance: 
The permutation [4, 3, 0, 7, 1, 8, 2, 9, 5, 6] results in a Makespan of 736
Solution after checking for acceptance: 
The permutation [4, 3, 0, 7, 1, 8, 2, 9, 5, 6] results in a Makespan of 736


In [15]:
print(f'SOlution before checking for acceptance: \n{newSolution}')

baseTemperature = 1 # Iterated Greedy第二个参数
currentBestObj = solver.SolutionPool.GetLowestMakespanSolution().Makespan 
# 上面这行表示，目前所有解决方案（solutionPool）中用时最短的方案的总时长。把这个最短的时间作为Obj

#"""新方案和当前方案的用时进行比较，若新方案更短，则接受这个方案"""
if newSolution.Makespan < currentSolution.Makespan:
    currentSolution = newSolution 

    #"""接受新方案只是第一步，因为新方案目前仅仅优于currentSolution，现在要判断newSolution是否为最优"""
    if newSolution.Makespan < currentBestObj:
        solver.SolutionPool.AddSolution(newSolution)
        currentBestObj = newSolution.Makespan

#"""若新方案并未没有提供任何优化，那么这里计算公式的结果，试图获取有多大概率接受这个新方案，即使新方案并不好"""
else:
    newObjValue = newSolution.Makespan
    currentObjValue = currentSolution.Makespan

    # 计算所有订单加工时间的总和-->公式中的分子
    totalProcessingTime = sum(x.ProcessingTime(i) for x in solver.InputData.InputJobs for i in range(len(x.Operations)))
    temperature = baseTemperature * totalProcessingTime / (solver.InputData.n * solver.InputData.m * 10)
    probability = math.exp(-(newObjValue - currentObjValue) / temperature)

    randomNumber = solver.RNG.random()
    if randomNumber < probability:
        currentSolution = newSolution

print(f'SOlution after checking for acceptance: \n {currentSolution}')


SOlution before checking for acceptance: 
The permutation [4, 3, 0, 7, 1, 8, 2, 9, 5, 6] results in a Makespan of 736
SOlution after checking for acceptance: 
 The permutation [4, 3, 0, 7, 1, 8, 2, 9, 5, 6] results in a Makespan of 736


Akzeptanzkriterium in Funktion für bessere Übersichtlichkeit

In [16]:
def AcceptWorseSolution(currentObjectiveValue, newObjectiveValue):  # 这个函数的参数表分别是 Cp''和Cp
    temperature = baseTemperature * solver.InputData.TotalProcessingTime / (solver.InputData.n * solver.InputData.m * 10)
    probability = math.exp(-(newObjectiveValue - currentObjectiveValue) / temperature)

    return solver.RNG.random() <= probability  # 返回一个逻辑值，为真时，正如函数名那样，我们要接受新方案

----
### 1.6 Lokale Suche
----
Verfahren der lokalen Suche können optional verwendet werden, um die aktuelle Lösung zu verbessern. Häufig wird __Iterative Improvement__ mit der Insertion-Nachbarschaft verwendet.

In [17]:
print(f'Solution before Local Search:\n{currentSolution}')

localSearch = IterativeImprovement(solver.InputData, 'BestImprovement', ['Insertion'])  # localscarch 只输出优化的结果，所以这里不用比较
localSearch.Initialize(solver.EvaluationLogic, solver.SolutionPool)
currentSolution = localSearch.Run(currentSolution)


print(f'Solution after nach Local Search\n{currentSolution}')

Solution before Local Search:
The permutation [4, 3, 0, 7, 1, 8, 2, 9, 5, 6] results in a Makespan of 736
Solution after nach Local Search
The permutation [1, 8, 4, 3, 0, 5, 2, 7, 9, 6] results in a Makespan of 713


In [18]:
print(f'Solution before LS: \n{currentSolution}')

localSearch = IterativeImprovement(solver.InputData, 'BestImprovement', ['Insertion'])
localSearch.Initialize(solver.EvaluationLogic, solver.SolutionPool)

currentSolution = localSearch.Run(currentSolution)

print(f'Solution after LS:\n{currentSolution}')

Solution before LS: 
The permutation [1, 8, 4, 3, 0, 5, 2, 7, 9, 6] results in a Makespan of 713
Solution after LS:
The permutation [1, 8, 4, 3, 0, 5, 2, 7, 9, 6] results in a Makespan of 713


----
### 1.7 Iterated Greedy als Steuerungsmechanismus der Komponenten
----
Iterated Greedy ist iterative Abfolge von Destruktion und Konstruktion $\Rightarrow$ wird in Schleife immer wieder durchgeführt bis Stoppkriterium erreicht ist. 

Hier im Beispiel ist Stoppkriterium die Anzahl an Iterationen _maxIterations_, aber bspw. auch Zeitlimit oder Anzahl Iterationen ohne Verbesserungen möglich.

In [19]:
maxIterations = 10 # stop criterion number of iterations

def Run(currentSolution):
    currentSolution = localSearch.Run(currentSolution)

    currentBestObj = solver.SolutionPool.GetLowestMakespanSolution().Makespan
    iteration = 0
    while iteration < maxIterations:   # while loop, because normally several termination criteria can be used
        removedJobs, partialPermutation = Destruction(currentSolution)

        newSolution = Construction(removedJobs, partialPermutation)

        newSolution = localSearch.Run(newSolution)

        if newSolution.Makespan < currentSolution.Makespan:
            currentSolution = newSolution

            if newSolution.Makespan < currentBestObj:  # newSolution he currentBastObj是什么类型的值
                print(f'new best solution found in iteration {iteration}: {currentSolution}')
                solver.SolutionPool.AddSolution(newSolution)
                currentBestObj = newSolution.Makespan

        elif AcceptWorseSolution(currentSolution.Makespan, newSolution.Makespan):
            currentSolution = newSolution

        iteration += 1

    return solver.SolutionPool.GetLowestMakespanSolution()



In [20]:
# 迭代的停止标准：总的迭代次数，即只要执行过IterGreedy，计数器就要加1。
#对于没有优化的迭代次数，可以理解为执行过AcceptWorseSolution的次数，即计数新方案不如当前方案的次数。
# AcceptWorseSolution返回一个bool值，当elif执行时，考虑到概率问题，这条语句执行的次数可能回很少，
# 因此无优化迭代的最大值通常较小
maxIterations = 10 

def Run(currentSolution):
    currentSolution = localSearch.Run(currentSolution)

    currentBestObj = solver.SolutionPool.GetLowestMakespanSolution().Makespan
    iteration = 0

    while iteration < maxIterations:
        removedJobs, partialPermutation = Destruction(currentSolution)
        newSolution = Construction(removedJobs, partialPermutation)

        newSolution = localSearch.Run(newSolution)

        if newSolution.Makespan < currentSolution.Makespan:
            currentSolution = newSolution

            if newSolution.Makespan < currentBestObj:
                print(f'new best solution found in iteration {iteration}: {currentSolution}')
                solver.SolutionPool.AddSolution(newSolution)
                currentBestObj = newSolution.Makespan

        elif AcceptWorseSolution(currentSolution.Makespan, newSolution.Makespan):
            currentSolution = newSolution

        iteration += 1

    return solver.SolutionPool.GetLowestMakespanSolution()

In [21]:
solver = Solver(data, 1060)

startSolution = solver.ConstructionPhase("NEH")

Generating an initial solution according to NEH.
Constructive solution found.
The permutation [4, 3, 7, 1, 8, 2, 9, 0, 5, 6] results in a Makespan of 752


Iterated Greedy ausführen

In [22]:
solver = Solver(data, 1060)

startSolution = solver.ConstructionPhase("NEH")

bestSolution = Run(startSolution)

print(f'Best solution found.\n{bestSolution}')  # 输出结果不一致，看看老师改了啥

Generating an initial solution according to NEH.
Constructive solution found.
The permutation [4, 3, 7, 1, 8, 2, 9, 0, 5, 6] results in a Makespan of 752
Best solution found.
The permutation [1, 8, 4, 3, 0, 5, 2, 7, 9, 6] results in a Makespan of 713


---
## 2. Klasse für Iterated Greedy
---

Analog wie für den Algorithmus _IterativeImprovement_ soll jetzt auch für _IteratedGreedy_ eine eigene Klasse angelegt werden, damit eine Instanz der Klasse an den Solver übergeben werden kann.

_Iterated Greedy_ soll wie _IterativeImprovement_ von _ImprovementAlgorithm_ erben.

Notwendige __Parameter__ sind Attribute:
* NumberJobsToRemove
* BaseTemperature
* MaxIterations
* Instanz des Local Search Algorithmus

EvaluationLogic, SolutionPool und random number generator werden von Solver an Algorithmus übergeben.

__Programmieraufgabe:__ Ergänzen sie die Klasse _IteratedGreedy_, sodass sie an den Solver als Algorithmus übergeben werden kann.

__Funktionen:__ Konstruktor, Initialize, Destruction, Construction, AcceptWorseSolution, Run

In [23]:
class IteratedGreedy(ImprovementAlgorithm):
    def __init__(self, inputData, numberJobsToRemove, baseTemperature, maxIterations, localSearchAlgorithm=None):
        super().__init__(inputData)

        self.NumberJobsToRemove = numberJobsToRemove
        self.BaseTemperature = baseTemperature
        self.MaxIterations = maxIterations
        
    def Destruction(self, solution): 
        # 思考：怎么随机删除？？？？
        removedJobs = self.RNG.choice(solver.InputData.n, size = self.NumberJobsToRemove, replace=False).tolist() # 哟放回和无放回

        partialPermutation = [ i for i in solution.Permutation if i not in removedJobs]

        return removedJobs, partialPermutation

    def Construction(self, removedJobs, partialPermutation):
        completeSolution = Solution(solver.InputData.InputJobs, partialPermutation)
        for i in removedJobs:
            solver.EvaluationLogic.DetermineBestInsertion(completeSolution, i) # best insertion in neh

        return completeSolution


    def AcceptWorseSolution(self, currentObjectiveValue, newObjectiveValue):  
        temperature = self.BaseTemperature * solver.InputData.TotalProcessingTime / (solver.InputData.n * solver.InputData.m * 10)
        probability = math.exp(-(newObjectiveValue - currentObjectiveValue) / temperature)

        return self.RNG.random() <= probability 

    def Run(self, currentSolution):
        currentSolution = localSearch.Run(currentSolution)

        currentBestObj = solver.SolutionPool.GetLowestMakespanSolution().Makespan
        iteration = 0

        while iteration < self.MaxIterations:
            removedJobs, partialPermutation = Destruction(currentSolution)
            newSolution = Construction(removedJobs, partialPermutation)

            newSolution = localSearch.Run(newSolution)

            if newSolution.Makespan < currentSolution.Makespan:
                currentSolution = newSolution

                if newSolution.Makespan < currentBestObj:
                    print(f'new best solution found in iteration {iteration}: {currentSolution}')
                    solver.SolutionPool.AddSolution(newSolution)
                    currentBestObj = newSolution.Makespan

            elif AcceptWorseSolution(currentSolution.Makespan, newSolution.Makespan):
                currentSolution = newSolution

            iteration += 1

        return solver.SolutionPool.GetLowestMakespanSolution()
  

Der Algorithmus kann jetzt in _RunLocalSearch()_ an den Solver übergeben und ausgeführt werden.

In [36]:
    data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json") #"../TestInstancesJson/Small/VFR40_10_3_SIST.json"
# 有些很大的数据，python处理起来会很慢，可以用Java

    # 这就是Iterated Greedy的localSearchAlgorithm参数
    insertion = IterativeImprovement(data, 'BestImprovement', ['Insertion'])
    # taillardInsertion = IterativeImprovement(data, 'BestImprovement', ['TaillardInsertion'])
    iteratedGreedy = IteratedGreedy( # IterateGreedy是一种优化算法，而solver的Run local search函数需要两个参数，第一个是启发式算法，第二个是优化算法
        data, 
        numberJobsToRemove=2, 
        baseTemperature=1, 
        maxIterations=100,
        localSearchAlgorithm=insertion)

    solver = Solver(data, 1010)

    # 在进行本地搜索前需要准备好用于优化启发式算法结果的优化算法。上一章讲的优化算法是创建邻域，通过插入和换位来获得优化。这一张讲的Iterated Greedy则是另一种思路，解构和重构
    solver.RunLocalSearch(constructiveSolutionMethod='NEH', algorithm=iteratedGreedy)

Generating an initial solution according to NEH.
Constructive solution found.
The permutation [4, 3, 7, 1, 8, 2, 9, 0, 5, 6] results in a Makespan of 752
Best found Solution.
The permutation [1, 8, 4, 3, 0, 5, 2, 7, 9, 6] results in a Makespan of 713


In [45]:
    data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json") #"../TestInstancesJson/Small/VFR40_10_3_SIST.json"
# 有些很大的数据，python处理起来会很慢，可以用Java

    # 这就是Iterated Greedy的localSearchAlgorithm参数
    insertion = IterativeImprovement(data, 'BestImprovement', ['Insertion'])
    # taillardInsertion = IterativeImprovement(data, 'BestImprovement', ['TaillardInsertion'])
    iteratedGreedy = IteratedGreedy( # IterateGreedy是一种优化算法，而solver的Run local search函数需要两个参数，第一个是启发式算法，第二个是优化算法
        data, 
        numberJobsToRemove=2, 
        baseTemperature=1, 
        maxIterations=100,
        localSearchAlgorithm=insertion)

    solver = Solver(data, 1010)

    bestInit = solver.ConstructionPhase('NEH')
    print(bestInit.Makespan)
    iteratedGreedy.Initialize(solver.EvaluationLogic, solver.SolutionPool, solver.RNG)

    solver.Initialize()
    best = iteratedGreedy.Run(bestInit)

    print(best.Makespan)

    # 

Generating an initial solution according to NEH.
Constructive solution found.
The permutation [4, 3, 7, 1, 8, 2, 9, 0, 5, 6] results in a Makespan of 752
752


AttributeError: 'Solver' object has no attribute 'OptimizationAlgorithm'

----
# 3. Exkurs: DEAP und Genetic Algorithms
----

Distributed Evolutionary Algorithms in Python ([DEAP](https://github.com/DEAP/deap)) ist ein Framework für Evolutionary Computation, mit u.a. den populationsbasierten Metaheuristiken Genetic Algorithms und Particle Swarm Optimization.

![Metaheuristiken](../DataFiles/MH.png)

### Wichtige Begriffe für Genetic Algorithms:
* Individuum: Einzelne Lösung
* Population: Sammlung von Individuen
* Generation: Population in einer Iteration
* Reproduktion: Erzeugung neuer Individuen aus bekannten Individuen
* Selektion: Auswahl von Individuen für Reproduktion und neue Generation
* Fitness: Bewertung eines Individuums (Zielfunktionswert)
* Mutation: Zufällige Veränderung eines Individuums

![GeneticAlgorithm](../DataFiles/GA.png)

----
### Beispiel DEAP
----

__Creator__

Meta-Factory um Klassen zu erzeugen

Parameters:
* name – The name of the class to create.
* base – A base class from which to inherit.
* attribute – One or more attributes to add on instantiation of this class, optional.


In [25]:
from Solver import *

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json")

solver = Solver(data, 1011)

creator.create("FitnessMin", base.Fitness, weights=[-1.0]) # minimization problem, weights as list，maximization问题，weight等于1
creator.create("Individual", list, fitness=creator.FitnessMin) # single individual stored in python list

__Toolbox__

Erzeugt Konfiguration des Algorithmus

In [26]:
toolbox = base.Toolbox() # list是怎么被填满的

# Individual is a permutation of integer indices
toolbox.register("indices", solver.RNG.choice, range(solver.InputData.n), solver.InputData.n, replace=False)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

# Population is a collection of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
pop = toolbox.population(100) # 100 individuals in population

# Operators
toolbox.register("mate", tools.cxPartialyMatched) # set crossover 
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05) # set mutation
toolbox.register("select", tools.selTournament, tournsize=3) # set selection mechanism

__Fitness-Funktion__

Bewertung eines Indivduums

In [27]:
def EvalPFSP(solver, individual):
    solution = Solution(solver.InputData.InputJobs, individual)
    solver.EvaluationLogic.DefineStartEnd(solution)
    return [solution.Makespan]

toolbox.register("evaluate", EvalPFSP, solver) # fitness function

__Statistiken__

Sammlung von Werten während Laufzeit des Algorithmus'

In [28]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

__Hall of fame__

solutionpool

Gedächtnis zum Speichern der besten Lösungen; hier nur beste Lösung

In [29]:
hof = tools.HallOfFame(1)

__Ausführen__

In [30]:
random.seed(solver.Seed) # ngen可以改，原本是100
algorithms.eaSimple(population=pop, toolbox=toolbox, cxpb=0.8, mutpb=0.2, ngen=500, stats=stats, halloffame=hof)

bestPermutation = hof[0]
bestSolution = Solution(solver.InputData.InputJobs, bestPermutation)
solver.EvaluationLogic.DefineStartEnd(bestSolution)
print(f'Best found Solution.\n {bestSolution}')

gen	nevals	avg   	std    	min	max
0  	100   	866.87	42.9324	757	985
1  	81    	829.45	42.7935	730	954
2  	82    	820.48	40.6065	743	915
3  	91    	809.32	36.6756	743	900
4  	81    	793.49	34.1264	743	894
5  	78    	784.84	36.3518	739	895
6  	85    	778.38	38.1512	738	921
7  	76    	768.52	30.2561	738	879
8  	90    	764.48	25.6743	738	871
9  	84    	769.89	31.5598	739	857
10 	84    	772.37	40.845 	736	950
11 	83    	766.35	34.4184	736	879
12 	89    	762.21	35.8484	726	952
13 	77    	757.56	30.777 	726	868
14 	90    	756.72	26.8652	726	840
15 	80    	757.71	30.4999	726	868
16 	88    	760.39	42.3719	726	1002
17 	82    	747.93	25.1862	726	851 
18 	90    	746.3 	29.4607	726	911 
19 	77    	737.24	15.9004	726	824 
20 	90    	732.04	10.3884	726	778 
21 	84    	730.92	17.4159	726	831 
22 	81    	731.35	18.3174	726	816 
23 	80    	728.35	11.0258	726	792 
24 	90    	728.12	12.4758	726	840 
25 	86    	728.48	13.3877	726	840 
26 	87    	728.66	11.2483	726	816 
27 	82    	730.1 	16.0253	726	819 
28

KeyboardInterrupt: 

----
#### Funktion für Solver
----

Parameter als Argumente
* Populationsgröße _populationSize_
* Anzahl Generationen _generations_
* Rekombinationswahrscheinlichkeit _matingProb_
* Mutationswahrscheinlichkeit _mutationProb_

In [ ]:
def EvalPFSP(self, individual):
    solution = Solution(self.InputData.InputJobs, individual)
    self.EvaluationLogic.DefineStartEnd(solution)
    return [solution.Makespan]

def RunGeneticAlgorithm(self, populationSize, generations, matingProb, mutationProb):
    # Creator - meta-factory to create new classes
    creator.create("FitnessMin", base.Fitness, weights=[-1.0])
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox() 

    # Individual is a permutation of integer indices
    toolbox.register("indices", solver.RNG.choice, range(self.InputData.n), self.InputData.n, replace=False)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

    # Population is a collection of individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    pop = toolbox.population(populationSize) # number of individuals in population

    # Operators
    toolbox.register("mate", tools.cxPartialyMatched) # set crossover 
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05) # set mutation
    toolbox.register("select", tools.selTournament, tournsize=3) # set selection mechanism
    
    # Fitness function
    toolbox.register("evaluate", self.EvalPFSP) 
    
    # Statistics during run time
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
            
    # Hall of fame --> Best individual
    hof = tools.HallOfFame(1)

    random.seed(self.Seed)
    algorithms.eaSimple(population=pop, toolbox=toolbox, cxpb=matingProb, mutpb=mutationProb, ngen=generations, stats=stats, halloffame=hof)

    bestPermutation = hof[0]
    bestSolution = Solution(self.InputData.InputJobs, bestPermutation)
    self.EvaluationLogic.DefineStartEnd(bestSolution)
    print(f'Best found Solution.\n {bestSolution}')

Solver.EvalPFSP = EvalPFSP
Solver.RunGeneticAlgorithm = RunGeneticAlgorithm

Vergleich Iterated Greedy und Genetic Algorithm

In [ ]:
data = InputData("../TestInstancesJson/Large/VFR100_20_1_SIST.json") # TestInstances/Small/VFR40_10_3_SIST.txt 
# 100 auftrag， 20 machine
taillardInsertion = IterativeImprovement(data, 'FirstImprovement', neighborhoodTypes=['TaillardInsertion'])
iteratedGreedy = IteratedGreedy(
    data, 
    numberJobsToRemove=2, 
    baseTemperature=1, 
    maxIterations=50, 
    localSearchAlgorithm=taillardInsertion)

solver = Solver(data, 1010)

solver.RunLocalSearch(
    constructiveSolutionMethod='NEH',
    algorithm=iteratedGreedy)
    
solver.RunGeneticAlgorithm(
    populationSize=100, 
    generations=200, 
    matingProb=0.8, 
    mutationProb=0.2)

# 学校电脑1份40秒，i5第三代，16G。我i7 6代，低压16G，6分钟

Generating an initial solution according to NEH.
Constructive solution found.
The permutation [61, 80, 51, 99, 11, 8, 62, 41, 24, 94, 67, 63, 0, 27, 31, 30, 82, 52, 91, 1, 19, 59, 89, 4, 49, 88, 25, 47, 85, 81, 38, 23, 5, 56, 71, 93, 83, 2, 64, 33, 96, 20, 21, 77, 78, 13, 60, 74, 98, 90, 6, 76, 9, 3, 42, 84, 50, 86, 95, 72, 97, 35, 40, 92, 15, 39, 10, 68, 43, 17, 26, 36, 18, 57, 44, 29, 54, 79, 55, 22, 48, 53, 75, 16, 73, 12, 58, 28, 32, 87, 70, 34, 7, 66, 45, 69, 14, 65, 46, 37] results in a Makespan of 6596
New best solution in iteration 2: The permutation [51, 61, 28, 80, 8, 62, 99, 31, 52, 63, 11, 94, 41, 67, 1, 82, 30, 91, 24, 4, 59, 47, 49, 88, 81, 85, 25, 38, 27, 23, 89, 71, 5, 56, 93, 83, 33, 96, 20, 77, 64, 17, 13, 19, 78, 60, 98, 90, 21, 6, 76, 43, 9, 3, 84, 86, 95, 40, 10, 35, 42, 74, 92, 15, 39, 97, 68, 26, 36, 18, 57, 0, 44, 29, 50, 54, 79, 55, 22, 48, 53, 75, 16, 73, 12, 58, 32, 87, 70, 34, 7, 66, 72, 45, 69, 14, 2, 65, 46, 37] results in a Makespan of 6424
New best solut

d:\ProgramData\Anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
d:\ProgramData\Anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg    	std    	min 	max 
0  	100   	7844.37	143.006	7500	8126
1  	78    	7767.61	149.492	7452	8199
2  	78    	7707.43	113.262	7346	7997
3  	76    	7689.86	134.223	7418	8138
4  	77    	7675.46	141.889	7425	8105
5  	90    	7666.71	141.713	7420	8145
6  	78    	7632.73	136.821	7411	8118
7  	86    	7605.7 	137.468	7371	7964
8  	82    	7596.6 	149.14 	7278	8035
9  	76    	7569.05	125.122	7278	7907
10 	81    	7549.81	136.695	7300	8057
11 	88    	7573.88	153.392	7262	8111
12 	87    	7535.16	135.361	7262	7859
13 	86    	7519.79	138.436	7221	7923
14 	87    	7531.76	157.896	7288	8240
15 	86    	7513.31	148.557	7270	7894
16 	85    	7466.2 	122.054	7243	7794
17 	71    	7480.18	135.351	7243	7901
18 	91    	7495.01	149.883	7256	7909
19 	85    	7493.16	149.326	7272	7939
20 	89    	7497.03	166.726	7263	8022
21 	86    	7471.71	141.29 	7216	7791
22 	92    	7471.26	134.687	7251	7797
23 	84    	7493.63	132.505	7256	7865
24 	93    	7491.55	134.574	7181	7903
25 	83    	7478.33	130.588	7195	7815
2